In [1]:
import os
import glob
import pandas as pd
import psycopg2 as pg2
from sql_queries import *

In [2]:
conn = pg2.connect(database = 'governmentpayroll', user = 'postgres', password = 'poop1234' )
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.csv'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

In [4]:
files = get_files('payroll')
files[0]

'C:\\Users\\alber\\Desktop\\myWork\\projects\\ETL\\payroll\\Payroll_Report_-_End_Date_1_23_2019.csv'

In [5]:
jan23 = pd.read_csv(files[0], dtype = object)
jan23.head()

,EMPLOYEE_NAME,OFFICE,CITY,EMPLOYEE_TITLE,BIWEEKLY/HOURLY_RATE,PAYROLL_TYPE,PAY_PERIOD,PAY_PERIOD_BEGIN_DATE,PAY_PERIOD_END_DATE,CHECK_DATE,LEGISLATIVE_ENTITY
0,"ABBOT, JUDITH L",SENATOR TOBY ANN STAVISKY,FLUSHING,COMMUNITY LIAISON,1076.93,SA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,SENATE EMPLOYEE
1,"ABRAHAM, PRINCY A",MINORITY COUNSEL/PROGRAM,ALBANY,ASSOCIATE COUNSEL,2376.93,RA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,SENATE EMPLOYEE
2,"ABREU, BRIAN A",LEGISLATIVE MESSENGER SERVICE,ALBANY,MESSENGER,1173.93,RA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,JOINT LEGISLATIVE EMPLOYEE
3,"ACEVEDO, SAMANTHA I",SENATOR JESSICA RAMOS,ALBANY,COMMITTEE DIRECTOR,2115.39,RA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,SENATE EMPLOYEE
4,"ADAMS, DAVID V",SENATE SERGEANT-AT-ARMS,ALBANY,SESSION SERGEANT-AT-ARMS,820.95,SA,21,01/10/2019 12:00:00 AM,01/23/2019 12:00:00 AM,02/06/2019 12:00:00 AM,SENATE EMPLOYEE


In [6]:
jan23['PAY_PERIOD_END_DATE'] = pd.to_datetime(jan23['PAY_PERIOD_END_DATE']).astype(object)
jan23['YEAR'] = jan23['PAY_PERIOD_END_DATE'].dt.year.astype(object)
payperiod_data = jan23.loc[0,['PAY_PERIOD', 'YEAR','PAY_PERIOD_BEGIN_DATE', 'PAY_PERIOD_END_DATE', 'CHECK_DATE']]
payperiod_data

PAY_PERIOD                                   21
YEAR                                       2019
PAY_PERIOD_BEGIN_DATE    01/10/2019 12:00:00 AM
PAY_PERIOD_END_DATE         2019-01-23 00:00:00
CHECK_DATE               02/06/2019 12:00:00 AM
Name: 0, dtype: object

In [ ]:
cur.execute(pay_period_table_insert, payperiod_data)
conn.commit()

In [7]:
payroll_type_data = jan23[['PAYROLL_TYPE']].drop_duplicates()
payroll_type_data.head()

,PAYROLL_TYPE
0,SA
1,RA
51,TE


In [ ]:
for i, row in payroll_type_data.iterrows():
    cur.execute(payroll_type_table_insert, row)
    conn.commit()

In [9]:
city_data = jan23[['CITY']].drop_duplicates()
city_data

,CITY
0,FLUSHING
1,ALBANY
5,BRONX
6,UTICA
7,NEW YORK
9,EAST ELMHURST
14,WILLIAMSVILLE
21,NEW CITY
25,ROCHESTER
26,BINGHAMTON


In [ ]:
for i, row in city_data.iterrows():
    cur.execute(city_table_insert, row)
    conn.commit()

In [10]:
legislative_entity_data = jan23[['LEGISLATIVE_ENTITY']].drop_duplicates()
legislative_entity_data

,LEGISLATIVE_ENTITY
0,SENATE EMPLOYEE
2,JOINT LEGISLATIVE EMPLOYEE


In [ ]:
for i, row in legislative_entity_data.iterrows():
    cur.execute(legislative_entity_insert, row)
    conn.commit()

In [ ]:
def isOneToOne(df, col1, col2):
    first = df.drop_duplicates([col1, col2]).groupby(col1)[col2].count().max()
    second = df.drop_duplicates([col1, col2]).groupby(col2)[col1].count().max()
    return first + second == 2

isOneToOne(jan_mod, 'LEGISLATIVE_ENTITY', 'EMPLOYEE_NAME')

first = jan23.groupby('PAY_PERIOD')['PAY_PERIOD_END_DATE'].count().max()
second = jan23.groupby('PAY_PERIOD_END_DATE')['PAY_PERIOD'].count().max()
print(first, second)